# 文本摘要

In [1]:
# !export https_proxy=http://127.0.0.1:7890 http_proxy=http://127.0.0.1:7890 all_proxy=socks5://127.0.0.1:7890

In [1]:
# %pip install pytorch_lightning
# %pip install rouge

In [2]:
from transformers import PegasusForConditionalGeneration
from tokenizers_pegasus import PegasusTokenizer

Building prefix dict from the default dictionary ...
Loading model from cache /Users/wangfiox/.cache/jieba.cache
Loading model cost 0.365 seconds.
Prefix dict has been built successfully.


In [3]:
model = PegasusForConditionalGeneration.from_pretrained(
    "IDEA-CCNL/Randeng-Pegasus-523M-Summary-Chinese-V1"
)

tokenizer = PegasusTokenizer.from_pretrained(
    "IDEA-CCNL/Randeng-Pegasus-523M-Summary-Chinese-V1"
)

In [5]:
print(model)

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(50002, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(50002, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [4]:
text = "在北京冬奥会自由式滑雪女子坡面障碍技巧决赛中，中国选手谷爱凌夺得银牌。祝贺谷爱凌！今天上午，自由式滑雪女子坡面障碍技巧决赛举行。决赛分三轮进行，取选手最佳成绩排名决出奖牌。第一跳，中国选手谷爱凌获得69.90分。在12位选手中排名第三。完成动作后，谷爱凌又扮了个鬼脸，甚是可爱。第二轮中，谷爱凌在道具区第三个障碍处失误，落地时摔倒。获得16.98分。网友：摔倒了也没关系，继续加油！在第二跳失误摔倒的情况下，谷爱凌顶住压力，第三跳稳稳发挥，流畅落地！获得86.23分！此轮比赛，共12位选手参赛，谷爱凌第10位出场。网友：看比赛时我比谷爱凌紧张，加油！"
inputs = tokenizer(
    text, max_length=1024, truncation=True, return_tensors="pt"
)  # 添加了 truncation=True

# Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
tokenizer.batch_decode(
    summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
)[0]

# model Output: 自由式滑雪女子坡面障碍技巧决赛谷爱凌摘银

'自由式滑雪女子坡面障碍技巧决赛谷爱凌摘银'

In [5]:
def summarize_text(text, max_length=1024, num_beams=4, length_penalty=2.0, max_length_output=150):
    """
    用输入的模型和分词器生成输入文本的摘要。
    参数：
        - text (str): 要生成摘要的输入文本。
    """
    
    inputs = tokenizer(text, max_length=max_length, truncation=True, return_tensors="pt")
    
    # Generate summary with beam search
    summary_ids = model.generate(
        inputs["input_ids"], 
        num_beams=num_beams, 
        length_penalty=length_penalty, 
        max_length=max_length_output, 
        no_repeat_ngram_size=3
    )
    
    return tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [6]:
text = """
【清深宣讲预报名】[欢兔喝彩]@所有人 清华大学深圳国际研究生院招生宣讲： 1. 计算机科学与技术项目介绍 2. 数据科学与技术项目介绍 3. 来自以上两个专业的师兄师姐分享申请经验 时间：6月16日（周五）下午 地点后续通知 各位同学，清华深研院本周五将面向我们学院的同学们进行宣讲，内容包含保研（含夏令营）及考研（他们考408）的相关信息。据了解，清深与我们专业相关的有两个计算机科学与技术和数据科学与技术两个方向，他们的学生需求很大，也表示非常喜欢我们的同学，欢迎同学们能够申请和报考。所以，有意向保研或考研到清深，或者还在观望的同学都能去看一看、听一听，多多了解。预报名链接：[清深宣讲预报名](https://hitsz.feishu.cn/share/base/form/shrcna5TjoEOpyrgWCqSt8E34sf)
"""
print(summarize_text(text))

清华大学深圳国际研究生院2016年6月16日清深宣讲预报名


对"通知内容"列进行概括

In [7]:
import pandas as pd
from tqdm import tqdm
import re

In [8]:
# 读取CSV文件
df = pd.read_csv("../data//dates_combined_data.csv")

# 创建一个新列来保存概括的内容
df["概括内容"] = ""

In [9]:
# 显示进度并更新DataFrame
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    summarized = summarize_text(row["通知内容"])
    df.at[index, "概括内容"] = summarized


 37%|███▋      | 114/307 [04:38<06:23,  1.99s/it]

In [ ]:
# 保存为新的CSV文件
df.to_csv("./dates_combined_data_summarized.csv", index=False)